In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [2]:
train_dir = "D:/4th 1st semester/computer vision/tasks/Alzheimer_s Dataset/Alzheimer_s Dataset/Alzheimer_s Dataset/train"
test_dir = "D:/4th 1st semester/computer vision/tasks/Alzheimer_s Dataset/Alzheimer_s Dataset/Alzheimer_s Dataset/test" 

In [3]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [4]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [5]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [6]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [7]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\Computec\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for LR
train_labels = train_generator.classes


  1/161 ━━━━━━━━━━━━━━━━━━━━ 29s 187ms/step

C:\Users\Computec\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 8s 47ms/step


In [9]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step


In [10]:
# Train Logistic Regression classifier
lr_classifier = LogisticRegression( 
    max_iter=1000,
    C=1.0,
    solver='lbfgs',
    penalty='l2')
lr_classifier.fit(train_features, train_labels)


LogisticRegression(max_iter=1000)

In [11]:
# Make predictions on the test data
test_predictions = lr_classifier.predict(test_features)

In [12]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 52.46%


In [13]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor3.h5')

In [14]:
# Save the trained Logistic Regression model
joblib.dump(lr_classifier, 'lr_classifier.pkl')

['lr_classifier.pkl']

In [16]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model("D:/4th 1st semester/computer vision/tasks/task4/task4.5/cnn_feature_extractor3.h5")
lr_classifier = joblib.load("D:/4th 1st semester/computer vision/tasks/task4/task4.5/lr_classifier.pkl")

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for lr

    # Classify the features with the lr model
    prediction = lr_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented',2: 'NonDemented',3:'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = "D:/4th 1st semester/computer vision/tasks/Alzheimer_s Dataset/Alzheimer_s Dataset/Alzheimer_s Dataset/test/NonDemented/26 (69).jpg"
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
The image is classified as: NonDemented
